In [15]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict
from pprint import pprint

In [16]:
essay= """The future of any nation is intrinsically linked to the sanctity of its childhood, and for India, home to the largest child population globally, the incidence of child maltreatment—a corrosive compound of abuse, neglect, and exploitation—represents not merely a humanitarian crisis but a profound systemic failure. Moving beyond a reductionist focus on individual criminality, a comprehensive analysis requires examining the multifactorial etiology of this vulnerability. The causes are deeply embedded in the intersection of socio-economic disparity, institutional lacunae, and entrenched cultural norms, demanding an urgent, holistic policy reckoning to safeguard the nation’s demographic dividend. The causality of child maltreatment is best understood through the Socio-Ecological Model, which delineates risk factors across four interdependent spheres, demonstrating that no single factor operates in isolation. At the apex of this model lies the normalisation of violence, where traditional, hierarchical societal structures often perpetuate a cultural acceptance of corporal punishment as a legitimate disciplinary tool, blurring the crucial ethical line between discipline and physical abuse. Furthermore, pervasive patriarchal norms diminish the agency of the child, particularly the girl child, rendering them susceptible to gendered forms of exploitation, including early marriage and sexual victimisation. Critically, structural poverty and widening economic inequality function as distal but powerful risk multipliers, generating chronic parental stress, financial precarity, and resource scarcity, which are strong precursors to emotional and physical neglect. The immediate civic environment often fails in its role as a secondary protective layer; community disinvestment, manifest in poor policing and the absence of safe public spaces, turns neighbourhoods into zones of peril. A critical failure lies in the dearth of accessible, high-quality support infrastructure, where the non-availability of affordable mental health services, de-addiction facilities, and formal parenting programmes leaves highly stressed or dysfunctional families without necessary safety valves, a form of institutional apathy that essentially pushes the private trauma of the home into the public reality of systemic neglect. The ultimate victimisation occurs at the Individual and Family levels, driven by deeply entrenched psychological and relational dynamics. A compelling and tragic cause is the Intergenerational Cycle of Abuse, where individuals who were harmed in their own childhood are statistically at a heightened risk of perpetrating similar harm, often rooted in underdeveloped emotional regulation skills and a skewed perception of nurturing. Furthermore, the pervasive challenge of substance abuse and untreated parental mental illness acts as a catastrophic disruption to the caregiving environment, leading to cognitive impairment, poor impulse control, and an inability to perceive or respond appropriately to the child’s needs. The family unit, intended as a bastion of security, often becomes the locus of danger, as domestic violence (Intimate Partner Violence) is inextricably linked to child maltreatment; a child witnessing parental violence suffers profound psychological trauma, classifying it as a severe form of emotional abuse. Social isolation and the absence of a strong kinship support system strip the primary caregiver of vital emotional and practical assistance, pushing them past their coping threshold. Addressing this multifaceted crisis requires a paradigm shift in state policy, moving from post-incident punitive response to proactive, evidence-based prevention. India possesses a robust legal framework, notably the Protection of Children from Sexual Offences (POCSO) Act, 2012, and the Juvenile Justice (Care and Protection of Children) Act, 2015, and the imperative now lies in rigorous, time-bound implementation. This necessitates the de-siloing of services, ensuring seamless coordination between Police, Child Welfare Committees (CWCs), and the Judiciary, coupled with the mandatory establishment of well-equipped Child-Friendly Courts to expedite trials and reinforce the deterrent effect of the law. True prevention must simultaneously tackle the root socio-economic causes: public policy, such as the Mission Vatsalya Scheme, must be leveraged to proactively identify high-risk families and deliver early intervention, including universal access to free positive parenting programmes and robust financial security nets to mitigate the impact of poverty-induced stress. Finally, a national mission is required to challenge the cultural legitimacy of corporal punishment through mass-media campaigns, alongside instituting mandatory training for all professionals in contact with children (teachers, doctors, Anganwadi workers) on recognising signs of maltreatment and adhering to statutory reporting protocols, thereby strengthening the institutional vigilance apparatus. Child vulnerability is a testament to the fact that laws alone are insufficient; their efficacy is contingent upon systemic commitment and societal transformation. India’s trajectory toward becoming a ‘Viksit Bharat’ (Developed India) is inseparable from its success in creating an environment where every child is protected from peril and assured of their fundamental rights to safety and dignity. A protected childhood is the sine qua non of a healthy democracy, and the State, in its parens patriae role, must undertake this challenge with unwavering political will, transforming its legislative framework into a living, responsive, and pervasive shield against all forms of harm."""

In [17]:
load_dotenv()
llm= ChatGoogleGenerativeAI(model= 'gemini-2.5-flash-lite')

In [18]:
class LLM_calling(TypedDict):
    language: int
    clarity_of_thaught: int
    depth_of_analysis: int
    text_feedback: str
    essay: str
    

In [19]:
def language_points(state: LLM_calling) -> LLM_calling:
    prompt= f'Evaluate the essay for language proficiency on a scale of 1 to 10 where 10 is the highest only in integer format. Essay: {state['essay']}'
    points= llm.invoke(prompt)
    return {'language': points}

def cot_points(state: LLM_calling) -> LLM_calling:
    prompt= f'Evaluate the essay for clarity of thoughts on a scale of 1 to 10 where 10 is the highest only in integer format. Essay: {state['essay']}'
    points= llm.invoke(prompt)
    return {'clarity_of_thaught': points}

def doa_points(state: LLM_calling) -> LLM_calling:
    prompt= f'Evaluate the essay for the depth of Analysis on a scale of 1 to 10 where 10 is the highest only in integer format. Essay: {state['essay']}'
    points= llm.invoke(prompt)
    return {'depth_of_analysis': points}

def feedback(state: LLM_calling) -> LLM_calling:
    prompt= f'Gereate a 50 words feedback from this Essay : {state['essay']} \n The points obtained in language , Clarity of Thoughts and Depth of Analysis are {state['language']} , {state['clarity_of_thaught']} , {state['depth_of_analysis']} respectively'
    points= llm.invoke(prompt)
    return {'text_feedback': points}

In [20]:
# Here cot and doa means clarity of thoughts and depth of analysis
graph= StateGraph(LLM_calling)
graph.add_node('language',language_points )
graph.add_node('cot',cot_points)
graph.add_node('doa',doa_points)
graph.add_node('feedback_summary', feedback)

graph.add_edge(START, 'language')
graph.add_edge(START,'cot')
graph.add_edge(START,'doa')

graph.add_edge('language','feedback_summary')
graph.add_edge('cot','feedback_summary')
graph.add_edge('doa','feedback_summary')
graph.add_edge('feedback_summary' , END)
workflow= graph.compile()

initial_graph= {'essay': essay}
final_graph= workflow.invoke(initial_graph)

In [ ]:
language_score= (int(final_graph['language'].content))
cot_score= (int(final_graph['clarity_of_thaught'].content))
doa_score= (int(final_graph['depth_of_analysis'].content))
avg_score= round((language_score + cot_score + doa_score)/3,2)
pprint(final_graph['text_feedback'].content)
pprint(f'Average Score Out of 10 is : {avg_score}')


('This essay offers a profound and comprehensive analysis of child '
 'maltreatment in India. Its clarity of thought and depth of understanding are '
 'exceptional, weaving together socio-economic, cultural, and psychological '
 'factors. The proposed solutions are insightful and actionable, demonstrating '
 'a clear vision for safeguarding childhood.')
'\n Average Score Out of 10 is : 9.67'
